IMPORT FILES

In [3]:
import os
os.getcwd()

'/Users/chaitanyatandon/Desktop/DREAM_11_v2/Dream-11/Code/Model_Final/Test'

In [5]:
import pandas as pd

file1_path = '../../../CSVs/Final/Test/career_test.csv'
file2_path = '../../../CSVs/Final/Test/recent_test.csv'
file3_path = '../../../CSVs/Final/Test/venue_test.csv'
file4_path = '../../../CSVs/Final/Test/player_match_data_test.csv'

df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)
df3 = pd.read_csv(file3_path)
df4 = pd.read_csv(file4_path)

In [7]:
print(df1.shape[0])
print(df2.shape[0])
print(df3.shape[0])
print(df4.shape[0])

64989
64989
64989
57153


In [6]:
# Perform the inner merge on 'match_id' and 'player_id' for all four DataFrames
merged_df = pd.merge(df1, df2, on=['match_id', 'player_id'], how='inner')
merged_df = pd.merge(merged_df, df3, on=['match_id', 'player_id'], how='inner')
merged_df = pd.merge(merged_df, df4, on=['match_id', 'player_id'], how='inner')

# The final DataFrame after all merges
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57153 entries, 0 to 57152
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   match_id                           57153 non-null  int64  
 1   player_id                          57153 non-null  object 
 2   career_batsman_100s_test           57153 non-null  int64  
 3   career_batsman_50s_test            57153 non-null  int64  
 4   career_batsman_average_runs_test   57153 non-null  float64
 5   career_batsman_total_runs_test     57153 non-null  int64  
 6   career_bowler_average_test         57153 non-null  float64
 7   career_bowler_wickets_test         57153 non-null  int64  
 8   career_fielder_total_catches_test  57153 non-null  int64  
 9   career_fielder_total_runouts_test  57153 non-null  int64  
 10  career_batsman_total_fours_test    57153 non-null  int64  
 11  career_batsman_total_sixes_test    57153 non-null  int

In [8]:
# Convert 'date' column to datetime
merged_df['date'] = pd.to_datetime(merged_df['date'])

# Split the DataFrame based on the condition (date <= June 30, 2024)
split_date = pd.to_datetime('2024-06-30')

merged_df_before_june = merged_df[merged_df['date'] <= split_date]
merged_df_after_june = merged_df[merged_df['date'] > split_date]



In [9]:
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler

# Drop columns that are not part of X_train
columns_to_drop = ['match_id', 'player_id', 'player_name', 'fantasy_points', 'team_name', 'role', 'date']
X_train = merged_df_before_june.drop(columns=columns_to_drop)

# Extract the target variable (fantasy_points)
y_train = merged_df_before_june['fantasy_points']

# # Standardize the features (mean=0, std=1)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)

# Initialize the Huber Regressor model
model = HuberRegressor()

# Fit the model to the scaled training data
model.fit(X_train, y_train)

# After training, you can check the coefficients and intercept
print(f"Model coefficients: {model.coef_}")
print(f"Model intercept: {model.intercept_}")


Model coefficients: [-3.65939217e-02  3.87464833e-03  9.47375624e-01 -1.64406949e-04
  7.51890307e-02  1.65166313e-02  1.59002418e-01  6.43801109e-02
 -1.50681907e-02 -2.78177357e-01  5.84259390e-02 -2.36049910e-01
  6.89740972e-04  1.00374092e-01 -6.09142912e-02 -1.13068874e-01
  2.77575071e-01  4.35651731e+00  8.36612994e-01  5.63782411e-02
 -1.58738406e-02  1.91604360e-02 -2.40626883e-02 -1.26630183e-02
  1.04004182e-01  4.28045711e-01  8.16429292e-02  8.53908166e-03
  4.78522672e-02 -3.80462896e-02]
Model intercept: 0.4201322762818633


/Users/chaitanyatandon/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [10]:
# X_test and y_test for merged_df_after_june
X_test = merged_df_after_june.drop(columns=columns_to_drop)
y_test = merged_df_after_june['fantasy_points']

# # Initialize the StandardScaler
# scaler = StandardScaler()


# # Transform X_test with the already fitted scaler
# X_test_scaled = scaler.transform(X_test)

# Use the trained model to make predictions on the test data
y_pred = model.predict(X_test)

# You can now compare y_pred with y_test to evaluate the model's performance


In [12]:

# Make predictions on X_test
y_pred = model.predict(X_test)

# Create a new DataFrame with the necessary columns
predictions_df = merged_df_after_june[['match_id', 'player_id', 'player_name', 'fantasy_points', 'role', 'team_name']].copy()

# Add the predicted fantasy points to the DataFrame
predictions_df['predicted_fantasy_points'] = y_pred

# Rename the fantasy_points column to actual_fantasy_points
predictions_df.rename(columns={'fantasy_points': 'actual_fantasy_points'}, inplace=True)

# Save the DataFrame to a CSV file
output_file = "Results/predicted_fantasy_points_odi.csv"  # You can change the file path as needed
predictions_df.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")


Predictions saved to Results/predicted_fantasy_points_odi.csv


In [13]:
import pandas as pd

# Input File (Predicted Fantasy Points)
input_file = "Results/predicted_fantasy_points_odi.csv"

# Output File
output_file = "Results/dream_team_with_predicted_fantasy_points.csv"

# Columns for Output
output_columns = [
    "match_id", "player_id", "player_name", "actual_fantasy_points", "role", "team_name", "predicted_fantasy_points"
]

# Read Data
data = pd.read_csv(input_file)

# Prepare Output Data
output_data = []

# Define Dream Team Calculation Logic
def calculate_dream_team(players):
    try:
        # Step 1: Pick one player from each role based on predicted fantasy points
        selected_players = []
        for role in ["Batsman", "Bowler", "Wicket-Keeper", "All-Rounder"]:
            role_players = [p for p in players if p["role"] == role]
            if role_players:
                selected_players.append(max(role_players, key=lambda x: x["predicted_fantasy_points"]))

        # Step 2: Select remaining players to make a team of 11
        remaining_players = [p for p in players if p not in selected_players]
        remaining_players.sort(key=lambda x: x["predicted_fantasy_points"], reverse=True)

        while len(selected_players) < 11:
            next_player = remaining_players.pop(0)
            selected_players.append(next_player)

        # Step 3: Ensure team diversity (max 5 players from one team)
        team_counts = pd.DataFrame(selected_players)["team_name"].value_counts()
        if team_counts.max() == 11:
            # Replace the lowest fantasy point player with the highest fantasy point player from the other team
            other_team = [team for team in players[0]["team_name"].unique() if team not in team_counts.index][0]
            lowest_fantasy_player = min(selected_players, key=lambda x: x["predicted_fantasy_points"])
            replacement_player = max(
                [p for p in players if p["team_name"] == other_team and p not in selected_players],
                key=lambda x: x["predicted_fantasy_points"]
            )
            selected_players.remove(lowest_fantasy_player)
            selected_players.append(replacement_player)

        # Step 4: Sort selected players by predicted fantasy points
        selected_players.sort(key=lambda x: x["predicted_fantasy_points"], reverse=True)

        return selected_players, "Optimal"

    except Exception as e:
        print(f"Error in calculating dream team: {e}")
        return None, "Error"

# Process Matches
i = 1
for match_id, match_group in data.groupby("match_id"):
    # Extract only necessary columns (player_id, role, team_name, predicted_fantasy_points)
    players = match_group[['match_id', 'player_id', 'player_name', 'actual_fantasy_points', 'role', 'team_name', 'predicted_fantasy_points']].to_dict(orient="records")
    
    # Calculate Dream Team
    selected_team, status = calculate_dream_team(players)

    if selected_team is None or len(selected_team) != 11:
        # If selected_team is None or doesn't have exactly 11 players, log the error
        with open("matches_where_team_could_not_be_formed.txt", 'a') as file:
            file.write(f"{status}, {match_id}, {len(selected_team) if selected_team else 'None'}, {selected_team if selected_team else 'None'}\n")
        i += 1
        continue

    # Prepare Row for Output
    for player in selected_team:
        row = [
            match_id,
            player["player_id"],
            player["player_name"],
            player["actual_fantasy_points"],
            player["role"],
            player["team_name"],
            player["predicted_fantasy_points"]
        ]
        output_data.append(row)

    if i % 100 == 0:
        print(f"Processed {i} matches...")
    i += 1

# Write Output to CSV
output_df = pd.DataFrame(output_data, columns=output_columns)
output_df.to_csv(output_file, index=False)

print(f"Dream Team details saved to {output_file}")



Dream Team details saved to Results/dream_team_with_predicted_fantasy_points.csv


In [14]:
import pandas as pd

# Input File (Predicted Fantasy Points)
input_file = "Results/predicted_fantasy_points_odi.csv"

# Output File
output_file = "Results/dream_team_with_actual_fantasy_points.csv"

# Columns for Output
output_columns = [
    "match_id", "player_id", "player_name", "actual_fantasy_points", "role", "team_name", "predicted_fantasy_points"
]

# Read Data
data = pd.read_csv(input_file)

# Prepare Output Data
output_data = []

# Define Dream Team Calculation Logic
def calculate_dream_team(players):
    try:
        # Step 1: Pick one player from each role based on predicted fantasy points
        selected_players = []
        for role in ["Batsman", "Bowler", "Wicket-Keeper", "All-Rounder"]:
            role_players = [p for p in players if p["role"] == role]
            if role_players:
                selected_players.append(max(role_players, key=lambda x: x["actual_fantasy_points"]))

        # Step 2: Select remaining players to make a team of 11
        remaining_players = [p for p in players if p not in selected_players]
        remaining_players.sort(key=lambda x: x["actual_fantasy_points"], reverse=True)

        while len(selected_players) < 11:
            next_player = remaining_players.pop(0)
            selected_players.append(next_player)

        # Step 3: Ensure team diversity (max 5 players from one team)
        team_counts = pd.DataFrame(selected_players)["team_name"].value_counts()
        if team_counts.max() == 11:
            # Replace the lowest fantasy point player with the highest fantasy point player from the other team
            other_team = [team for team in players[0]["team_name"].unique() if team not in team_counts.index][0]
            lowest_fantasy_player = min(selected_players, key=lambda x: x["actual_fantasy_points"])
            replacement_player = max(
                [p for p in players if p["team_name"] == other_team and p not in selected_players],
                key=lambda x: x["actual_fantasy_points"]
            )
            selected_players.remove(lowest_fantasy_player)
            selected_players.append(replacement_player)

        # Step 4: Sort selected players by actual fantasy points
        selected_players.sort(key=lambda x: x["actual_fantasy_points"], reverse=True)

        return selected_players, "Optimal"

    except Exception as e:
        print(f"Error in calculating dream team: {e}")
        return None, "Error"

# Process Matches
i = 1
for match_id, match_group in data.groupby("match_id"):
    # Extract only necessary columns (player_id, role, team_name, predicted_fantasy_points)
    players = match_group[['match_id', 'player_id', 'player_name', 'actual_fantasy_points', 'role', 'team_name', 'predicted_fantasy_points']].to_dict(orient="records")
    
    # Calculate Dream Team
    selected_team, status = calculate_dream_team(players)

    if selected_team is None or len(selected_team) != 11:
        # If selected_team is None or doesn't have exactly 11 players, log the error
        with open("matches_where_team_could_not_be_formed.txt", 'a') as file:
            file.write(f"{status}, {match_id}, {len(selected_team) if selected_team else 'None'}, {selected_team if selected_team else 'None'}\n")
        i += 1
        continue

    # Prepare Row for Output
    for player in selected_team:
        row = [
            match_id,
            player["player_id"],
            player["player_name"],
            player["actual_fantasy_points"],
            player["role"],
            player["team_name"],
            player["predicted_fantasy_points"]
        ]
        output_data.append(row)

    if i % 100 == 0:
        print(f"Processed {i} matches...")
    i += 1

# Write Output to CSV
output_df = pd.DataFrame(output_data, columns=output_columns)
output_df.to_csv(output_file, index=False)

print(f"Dream Team details saved to {output_file}")



Dream Team details saved to Results/dream_team_with_actual_fantasy_points.csv


In [15]:
import pandas as pd

# File paths for the input files
predicted_file = "Results/dream_team_with_predicted_fantasy_points.csv"
actual_file = "Results/dream_team_with_actual_fantasy_points.csv"

# Output file path
output_file = "Results/dream_team_summary.csv"

# Read the data from the two files
predicted_df = pd.read_csv(predicted_file)
actual_df = pd.read_csv(actual_file)

# Initialize list to store the summary for each match_id
summary_data = []

# Function to calculate the sum of fantasy points (with captain and vice-captain multipliers)
def calculate_team_points(team_df):
    team_df = team_df.sort_values(by='predicted_fantasy_points', ascending=False)  # Sort by predicted points (or actual, depending on case)
    
    total_points = 0
    if len(team_df) >= 1:
        # Highest points player (Captain)
        total_points += team_df.iloc[0]['predicted_fantasy_points'] * 2
    if len(team_df) >= 2:
        # Second highest points player (Vice-Captain)
        total_points += team_df.iloc[1]['predicted_fantasy_points'] * 1.5
    
    # Add the rest normally
    total_points += team_df.iloc[2:]['predicted_fantasy_points'].sum()

    return total_points

# Iterate through the unique match IDs
for match_id in predicted_df['match_id'].unique():
    # Get the predicted dream team for the current match_id
    predicted_team = predicted_df[predicted_df['match_id'] == match_id]

    # Get the actual dream team for the current match_id
    actual_team = actual_df[actual_df['match_id'] == match_id]

    # Calculate the sum of actual and predicted points for both teams
    predicted_team_points = calculate_team_points(predicted_team)
    actual_team_points_predicted = calculate_team_points(actual_team)
    
    # Add the match summary to the summary_data list
    summary_data.append({
        'match_id': match_id,
        'sum_predicted_points_predicted_team': predicted_team_points,
        'sum_predicted_points_actual_team': actual_team_points_predicted,
        'sum_actual_points_predicted_team': predicted_team['actual_fantasy_points'].sum(),
        'sum_actual_points_actual_team': actual_team['actual_fantasy_points'].sum()
    })

# Create a DataFrame from the summary_data
summary_df = pd.DataFrame(summary_data)

# Write the summary data to a CSV file
summary_df.to_csv(output_file, index=False)

print(f"Summary saved to {output_file}")


Summary saved to Results/dream_team_summary.csv


In [16]:
import pandas as pd

# Load the dream_team_summary.csv file into a DataFrame
summary_file = "Results/dream_team_summary.csv"
df = pd.read_csv(summary_file)

# Calculate the percentage errors for each match
df['%error_predicted_actual'] = abs(df['sum_actual_points_predicted_team'] - df['sum_actual_points_actual_team']) / df['sum_actual_points_actual_team'] * 100
df['%error_predicted_predicted'] = abs(df['sum_predicted_points_predicted_team'] - df['sum_actual_points_actual_team']) / df['sum_actual_points_actual_team'] * 100

# Print the results with the percentage errors
print("Percentage Errors for each match:")
print(df[['match_id', '%error_predicted_actual', '%error_predicted_predicted']])

# Calculate the mean of the percentage errors
mean_error_predicted_actual = df['%error_predicted_actual'].mean()
mean_error_predicted_predicted = df['%error_predicted_predicted'].mean()

# Print the mean errors
print(f"\nMean % error between predicted team actual points and actual team actual points: {mean_error_predicted_actual:.2f}%")
print(f"Mean % error between predicted team predicted points and actual team actual points: {mean_error_predicted_predicted:.2f}%")


Percentage Errors for each match:
    match_id  %error_predicted_actual  %error_predicted_predicted
0    1385691                18.826937                    2.096082
1    1385692                23.927239                   41.251034
2    1385693                25.263811                   13.792099
3    1385694                23.571040                   24.612990
4    1385695                12.400398                   30.559202
..       ...                      ...                         ...
74   1447097                47.494305                    8.336413
75   1452137                13.018868                   30.358557
76   1452138                25.527535                   33.685621
77   1453293                14.140271                   20.702089
78   1453294                17.110036                   19.215252

[79 rows x 3 columns]

Mean % error between predicted team actual points and actual team actual points: 27.20%
Mean % error between predicted team predicted points and actua

In [17]:
import pandas as pd

# Load the dream_team_summary.csv file into a DataFrame
summary_file = "Results/dream_team_summary.csv"
df = pd.read_csv(summary_file)

# Calculate the percentage errors for each match
df['%error_predicted_actual'] = abs(df['sum_actual_points_predicted_team'] - df['sum_actual_points_actual_team']) / df['sum_actual_points_actual_team'] * 100
df['%error_predicted_predicted'] = abs(df['sum_predicted_points_predicted_team'] - df['sum_actual_points_actual_team']) / df['sum_actual_points_actual_team'] * 100

# Calculate the mean of the percentage errors
mean_error_predicted_actual = df['%error_predicted_actual'].mean()
mean_error_predicted_predicted = df['%error_predicted_predicted'].mean()

# Print the results with the percentage errors
print("Percentage Errors for each match:")
print(df[['match_id', '%error_predicted_actual', '%error_predicted_predicted']])

# Print the mean errors
print(f"\nMean % error between predicted team actual points and actual team actual points: {mean_error_predicted_actual:.2f}%")
print(f"Mean % error between predicted team predicted points and actual team actual points: {mean_error_predicted_predicted:.2f}%")

# Save the updated DataFrame to the same CSV file
df.to_csv(summary_file, index=False)

# Optionally, save to a new file if you don't want to overwrite the original
# df.to_csv("Results/dream_team_summary_with_errors.csv", index=False)


Percentage Errors for each match:
    match_id  %error_predicted_actual  %error_predicted_predicted
0    1385691                18.826937                    2.096082
1    1385692                23.927239                   41.251034
2    1385693                25.263811                   13.792099
3    1385694                23.571040                   24.612990
4    1385695                12.400398                   30.559202
..       ...                      ...                         ...
74   1447097                47.494305                    8.336413
75   1452137                13.018868                   30.358557
76   1452138                25.527535                   33.685621
77   1453293                14.140271                   20.702089
78   1453294                17.110036                   19.215252

[79 rows x 3 columns]

Mean % error between predicted team actual points and actual team actual points: 27.20%
Mean % error between predicted team predicted points and actua

SAVING MODEL

In [18]:
import joblib



# Save the model to a file
joblib.dump(model, 'model_odi.pkl')


['model_odi.pkl']